## compute model derived fields (vorticity, steric height)

This notebook can be used to compute steric height and vorticity for regional llc4320 output.

Output is stored in new netCDF files in a "derived" directory.

Requires that an Argo climatology has been downloaded and stored locally.

NOTE: Suggested changes:
- computing strain, MLD, or other parameters
- making code work with other simulation data
- turned this notebook into a script that is called by the parent notebook
- loading the reference T/S data from ERDDAP (or another gridded Argo database)

Initial commit Aug 9, 2021 by kdrushka

In [1]:
%matplotlib inline
import os
# import sys
# import fsspec
import numpy as np
import glob
# import re
import gsw as sw
import xarray as xr
import dask.array as dsa
import xgcm.grid
import netCDF4 as nc4


In [2]:
pwd


'/home/manjaree/.ssh/oceanliner/testing'

In [3]:
cd /home/manjaree/.ssh/oceanliner/testing

/home/manjaree/.ssh/oceanliner/testing


In [4]:
ls -l


total 8084
-rw-rw-r-- 1 manjaree manjaree    6368 Jul 14 08:46 argo_local.nc
-rw-rw-r-- 1 manjaree manjaree  156268 Jul 18 16:41 compute_model_derived_fields.ipynb
drwxrwxr-x 2 manjaree manjaree    4096 Jul 14 08:46 dask-worker-space/
-rw-rw-r-- 1 manjaree manjaree  406468 Jul 14 08:46 demo.ipynb
-rw-rw-r-- 1 manjaree manjaree  853703 Jul 14 08:46 download_llc4320.ipynb
-rw-rw-r-- 1 manjaree manjaree  468805 Jul 14 08:46 final_project_updated.ipynb
-rw-rw-r-- 1 manjaree manjaree 4254897 Jul 14 08:46 get_swot_simulated_data.ipynb
drwxrwxr-x 2 manjaree manjaree    4096 Jul 14 08:46 myData/
-rw-rw-r-- 1 manjaree manjaree   66882 Jul 14 08:46 osse_code_cloud.ipynb
-rw-rw-r-- 1 manjaree manjaree  504415 Jul 14 08:46 osse_code.ipynb
-rw-rw-r-- 1 manjaree manjaree  425432 Jul 14 08:46 OSSE_code_mb.ipynb
-rw-rw-r-- 1 manjaree manjaree   47313 Jul 14 08:46 osse_tools_cloud.py
-rw-rw-r-- 1 manjaree manjaree   46215 Jul 14 08:46 osse_tools.py
-rw-rw-r-- 1 manjaree manjaree   60927 Jul 14 08:46 sn

In [5]:
llc4320dir = '/home/manjaree/Documents/LLC4320_pre-SWOT_10_days/ACC_SMST/osse_model_input/'
llc4320dir 

'/home/manjaree/Documents/LLC4320_pre-SWOT_10_days/ACC_SMST/osse_model_input/'

In [6]:
# --------------------------------------------------------------------
# USER INPUTS:
# specify region from this list:
# WesternMed  ROAM_MIZ  NewCaledonia  NWPacific  BassStrait  RockallTrough  ACC_SMST
# MarmaraSea  LabradorSea  CapeBasin
RegionName = 'ACC_SMST' 

# directory where model data is stored:
#llc4320dir = '/data1/adac/mitgcm/netcdf/'
llc4320dir = '/home/manjaree/Documents/LLC4320_pre-SWOT_10_days/ACC_SMST/osse_model_input/'
regiondir = '/home/manjaree/Documents/LLC4320_pre-SWOT_10_days/ACC_SMST/osse_model_input/'
#regiondir = llc4320dir + RegionName + '/'
# directory to save derived data to - create if doesn't exist
#derivedir = regiondir + 'derived/'
derivedir = '/home/manjaree/Documents/LLC4320_pre-SWOT_10_days/ACC_SMST/derived/'
if not(os.path.isdir(derivedir)):
    os.mkdir(derivedir)

In [7]:
# load a single file to get coordinates
fg = sorted(glob.glob(regiondir + '**nc')) # all files
i=0
thisf=fg[i]
print(thisf)
ds = xr.open_dataset(thisf)
    
# mean lat/lon of domain
xav = ds.XC.isel(j=0).mean(dim='i')
yav = ds.YC.isel(i=0).mean(dim='j')
print('center of domain: ', yav.values, ',' , xav.values)

/home/manjaree/Documents/LLC4320_pre-SWOT_10_days/ACC_SMST/osse_model_input/LLC4320_pre-SWOT_ACC_SMST_20120101.nc
center of domain:  -55.282402 , 153.0


In [8]:
# for vorticity calculation, build the xgcm grid:
# see https://xgcm.readthedocs.io/en/latest/xgcm-examples/02_mitgcm.html
grid = xgcm.Grid(ds, coords={'X':{'center': 'i', 'left': 'i_g'}, 
                 'Y':{'center': 'j', 'left': 'j_g'},
                 'T':{'center': 'time'},
                 'Z':{'center': 'k'}})

In [9]:
grid

<xgcm.Grid>
X Axis (periodic, boundary=None):
  * center   i --> left
  * left     i_g --> center
Y Axis (periodic, boundary=None):
  * center   j --> left
  * left     j_g --> center
T Axis (periodic, boundary=None):
  * center   time
Z Axis (periodic, boundary=None):
  * center   k

In [10]:
# load reference file of argo data
# NOTE: could update to pull from ERDDAP or similar
#argoclimfile = '/data1/argo/argo_CLIM_3x3.nc'
argoclimfile = '/home/manjaree/.ssh/oceanliner/testing/argo_local.nc'
argods = xr.open_dataset(argoclimfile,decode_times=False) 
# reference profiles: annual average Argo T/S using nearest neighbor
Tref = argods["temp"].sel(latitude=yav,longitude=xav, method='nearest').mean(dim='time')
Sref = argods["salt"].sel(latitude=yav,longitude=xav, method='nearest').mean(dim='time')
# SA and CT from gsw:
# see example from https://discourse.pangeo.io/t/wrapped-for-dask-teos-10-gibbs-seawater-gsw-oceanographic-toolbox/466
Pref = xr.apply_ufunc(sw.p_from_z, -argods.LEV, yav)
Pref.compute()
SAref = xr.apply_ufunc(sw.SA_from_SP, Sref, Pref, xav, yav,
                       dask='parallelized', output_dtypes=[Sref.dtype])
SAref.compute()
CTref = xr.apply_ufunc(sw.CT_from_pt, Sref, Tref, # Theta is potential temperature
                       dask='parallelized', output_dtypes=[Sref.dtype])
CTref.compute()
Dref = xr.apply_ufunc(sw.density.rho, SAref, CTref, Pref,
                    dask='parallelized', output_dtypes=[Sref.dtype])
Dref.compute()
print()

In [11]:
Dref

<xarray.DataArray (LEV: 27)>
array([1026.84210864, 1026.86576986, 1026.88932413, 1026.93630681,
       1026.9810488 , 1027.08184499, 1027.21545103, 1027.42689962,
       1027.62863245, 1027.78323574, 1028.032685  , 1028.39793937,
       1028.6047681 , 1029.18590338, 1029.7443514 , 1030.29343911,
       1030.81950379, 1031.34419174, 1031.85706082, 1032.36120735,
       1032.85426514, 1033.34894085, 1033.83459135, 1034.31597463,
       1034.79590148, 1035.97796239,           nan])
Coordinates:
  * LEV        (LEV) float64 0.0 5.0 10.0 20.0 ... 1.5e+03 1.75e+03 2e+03
    latitude   float64 -55.5
    longitude  float64 152.5
    i          float32 0.0
    j          float32 0.0

In [12]:
 # create datasets for variables of interest:
ss = ds.Salt
tt = ds.Theta
pp = xr.DataArray(sw.p_from_z(ds.Z,ds.YC))

In [13]:
%%time
# 1. compute absolute salinity and conservative temperature
sa = xr.apply_ufunc(sw.SA_from_SP, ss, pp, xav, yav, output_dtypes=[ss.dtype])
sa.compute()
ct = xr.apply_ufunc(sw.CT_from_pt, sa, tt, output_dtypes=[ss.dtype])
ct.compute()
dd = xr.apply_ufunc(sw.density.rho, sa, ct, pp, output_dtypes=[ss.dtype])
dd.compute()

CPU times: user 1min 6s, sys: 2.42 s, total: 1min 9s
Wall time: 1min 11s


<xarray.DataArray (time: 24, k: 88, j: 395, i: 480)>
array([[[[1026.99810471, 1026.99895428, 1026.99853964, ...,
          1026.91353114, 1026.91628012, 1026.92135968],
         [1026.99705223, 1026.99749117, 1026.99697162, ...,
          1026.91330223, 1026.91605806, 1026.92083723],
         [1026.99609331, 1026.99628904, 1026.99554935, ...,
          1026.91315091, 1026.91588582, 1026.92045442],
         ...,
         [1026.87755162, 1026.87855173, 1026.8778911 , ...,
          1026.74122345, 1026.73705124, 1026.73326929],
         [1026.86685087, 1026.87400464, 1026.87682309, ...,
          1026.72847774, 1026.72279875, 1026.71848386],
         [1026.85282269, 1026.86056252, 1026.86961327, ...,
          1026.72041987, 1026.71528321, 1026.71160152]],

        [[1027.00417098, 1027.00499685, 1027.00455082, ...,
          1026.91813688, 1026.92093395, 1026.92604663],
         [1027.00312466, 1027.00354463, 1027.00299046, ...,
          1026.91790422, 1026.92070707, 1026.92551758],
         [1027.00217048, 1027.00234634, 1027.00158063, ...,
          1026.91774991, 1026.92053293, 1026.92513623],
...
         [          nan,           nan,           nan, ...,
                    nan,           nan,           nan],
         [          nan,           nan,           nan, ...,
                    nan,           nan,           nan],
         [          nan,           nan,           nan, ...,
                    nan,           nan,           nan]],

        [[          nan,           nan,           nan, ...,
                    nan,           nan,           nan],
         [          nan,           nan,           nan, ...,
                    nan,           nan,           nan],
         [          nan,           nan,           nan, ...,
                    nan,           nan,           nan],
         ...,
         [          nan,           nan,           nan, ...,
                    nan,           nan,           nan],
         [          nan,           nan,           nan, ...,
                    nan,           nan,           nan],
         [          nan,           nan,           nan, ...,
                    nan,           nan,           nan]]]])
Coordinates:
  * i        (i) float32 0.0 1.0 2.0 3.0 4.0 ... 475.0 476.0 477.0 478.0 479.0
  * j        (j) float32 0.0 1.0 2.0 3.0 4.0 ... 390.0 391.0 392.0 393.0 394.0
  * k        (k) int32 0 1 2 3 4 5 6 7 8 9 10 ... 78 79 80 81 82 83 84 85 86 87
  * time     (time) datetime64[ns] 2012-01-01 ... 2012-01-01T23:00:00

In [14]:
%%time
# 2. compute specific volume anomaly: gsw.density.specvol_anom_standard(SA, CT, p)
sva = xr.apply_ufunc(sw.density.specvol_anom_standard, sa, ct, pp, output_dtypes=[ss.dtype])
sva.compute()

CPU times: user 12 s, sys: 236 ms, total: 12.2 s
Wall time: 12.2 s


<xarray.DataArray (time: 24, k: 88, j: 395, i: 480)>
array([[[[1.05251985e-06, 1.05171436e-06, 1.05210749e-06, ...,
          1.13271187e-06, 1.13010510e-06, 1.12528835e-06],
         [1.05351772e-06, 1.05310155e-06, 1.05359415e-06, ...,
          1.13292894e-06, 1.13031566e-06, 1.12578376e-06],
         [1.05442689e-06, 1.05424131e-06, 1.05494262e-06, ...,
          1.13307242e-06, 1.13047899e-06, 1.12614676e-06],
         ...,
         [1.16683050e-06, 1.16588206e-06, 1.16650857e-06, ...,
          1.29613272e-06, 1.30009044e-06, 1.30367800e-06],
         [1.17697852e-06, 1.17019425e-06, 1.16752140e-06, ...,
          1.30822330e-06, 1.31361049e-06, 1.31770371e-06],
         [1.19028243e-06, 1.18294216e-06, 1.17435878e-06, ...,
          1.31586716e-06, 1.32073995e-06, 1.32423255e-06]],

        [[1.05163444e-06, 1.05085143e-06, 1.05127432e-06, ...,
          1.13321047e-06, 1.13055813e-06, 1.12571002e-06],
         [1.05262646e-06, 1.05222828e-06, 1.05275369e-06, ...,
          1.13343109e-06, 1.13077327e-06, 1.12621168e-06],
         [1.05353112e-06, 1.05336438e-06, 1.05409035e-06, ...,
          1.13357741e-06, 1.13093838e-06, 1.12657329e-06],
...
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan]],

        [[           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         ...,
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan]]]])
Coordinates:
  * i        (i) float32 0.0 1.0 2.0 3.0 4.0 ... 475.0 476.0 477.0 478.0 479.0
  * j        (j) float32 0.0 1.0 2.0 3.0 4.0 ... 390.0 391.0 392.0 393.0 394.0
  * k        (k) int32 0 1 2 3 4 5 6 7 8 9 10 ... 78 79 80 81 82 83 84 85 86 87
  * time     (time) datetime64[ns] 2012-01-01 ... 2012-01-01T23:00:00

In [15]:
%%time
# 3. compute steric height = integral(0:z1) of Dref(z)*sva(z)*dz(z)
# - first, interpolate Dref to the model pressure levels
Drefi = Dref.interp(LEV=-ds.Z)
dz = -ds.Z_bnds.diff(dim='nb').drop_vars('nb').squeeze() # distance between interfaces

# steric height computation (summation/integral)
# - increase the size of Drefi and dz to match the size of sva
Db = Drefi.broadcast_like(sva)
dzb = dz.broadcast_like(sva)
dum = Db * sva * dzb
sh = dum.cumsum(dim='k')

CPU times: user 3.04 s, sys: 992 ms, total: 4.03 s
Wall time: 4.05 s


In [16]:
%%time
# --- compute vorticity using xgcm and interpolate to X, Y
        # see https://xgcm.readthedocs.io/en/latest/xgcm-examples/02_mitgcm.htm
vorticity = (grid.diff(ds.V*ds.DXG, 'X') - grid.diff(ds.U*ds.DYG, 'Y'))/ds.RAZ
#vorticity = grid.interp(grid.interp(vorticity, 'X', boundary='extend'), 'Y', boundary='extend')


CPU times: user 18.6 s, sys: 8.4 s, total: 27 s
Wall time: 30.7 s


In [17]:
vorticity

<xarray.DataArray (time: 24, k: 88, j_g: 395, i_g: 480)>
array([[[[ 1.54255059e-05, -1.34049461e-04, -1.25532173e-04, ...,
           2.55901738e-04,  2.83329819e-04,  3.23773538e-04],
         [ 9.81727298e-05, -5.27835727e-05, -5.27983235e-05, ...,
          -6.73247765e-06,  8.72679448e-06,  1.81029358e-05],
         [ 8.44423195e-05, -5.52797861e-05, -4.46054705e-05, ...,
          -3.83243162e-07,  1.29714219e-05,  1.72573393e-05],
         ...,
         [-1.59809258e-04, -1.16834464e-05,  4.40490045e-06, ...,
           2.22516344e-05,  2.76656620e-05,  2.96869599e-05],
         [-1.28116123e-04, -2.85566231e-05, -1.64094306e-05, ...,
           2.00648438e-05,  2.47191522e-05,  2.96625305e-05],
         [-7.81664445e-05,  6.01115773e-06, -2.24209919e-05, ...,
           1.74382776e-05,  2.04893742e-05,  2.47661187e-05]],

        [[ 4.81566263e-05, -1.35041863e-04, -1.26251443e-04, ...,
           2.46754523e-04,  2.74552043e-04,  3.15141784e-04],
         [ 1.31501714e-04, -5.30366182e-05, -5.29585456e-05, ...,
          -6.79180060e-06,  8.70062785e-06,  1.80822074e-05],
         [ 1.17486588e-04, -5.54086139e-05, -4.44819715e-05, ...,
          -4.82181650e-07,  1.29677649e-05,  1.71946584e-05],
...
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan]],

        [[            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         ...,
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan]]]])
Coordinates:
  * time     (time) datetime64[ns] 2012-01-01 ... 2012-01-01T23:00:00
  * k        (k) int32 0 1 2 3 4 5 6 7 8 9 10 ... 78 79 80 81 82 83 84 85 86 87
  * j_g      (j_g) float32 0.0 1.0 2.0 3.0 4.0 ... 390.0 391.0 392.0 393.0 394.0
  * i_g      (i_g) float32 0.0 1.0 2.0 3.0 4.0 ... 475.0 476.0 477.0 478.0 479.0

In [18]:
%%time
# --- compute vorticity using xgcm and interpolate to X, Y
        # see https://xgcm.readthedocs.io/en/latest/xgcm-examples/02_mitgcm.htm
#vorticity = (grid.diff(ds.V*ds.DXG, 'X') - grid.diff(ds.U*ds.DYG, 'Y'))/ds.RAZ
vorticity = grid.interp(vorticity, 'X', boundary='extend')

CPU times: user 1.29 s, sys: 4.69 s, total: 5.98 s
Wall time: 6.01 s


In [19]:
vorticity

<xarray.DataArray (time: 24, k: 88, j_g: 395, i: 480)>
array([[[[-5.93119777e-05, -1.29790817e-04, -1.22231174e-04, ...,
           2.69615779e-04,  3.03551678e-04,  1.69599522e-04],
         [ 2.26945785e-05, -5.27909481e-05, -4.84789004e-05, ...,
           9.97158414e-07,  1.34148651e-05,  5.81378328e-05],
         [ 1.45812667e-05, -4.99426283e-05, -3.88204595e-05, ...,
           6.29408937e-06,  1.51143806e-05,  5.08498294e-05],
         ...,
         [-8.57463521e-05, -3.63927296e-06,  3.46532682e-06, ...,
           2.49586482e-05,  2.86763109e-05, -6.50611490e-05],
         [-7.83363731e-05, -2.24830269e-05, -1.06221595e-05, ...,
           2.23919980e-05,  2.71908414e-05, -4.92267963e-05],
         [-3.60776434e-05, -8.20491706e-06, -2.83065076e-05, ...,
           1.89638259e-05,  2.26277465e-05, -2.67001629e-05]],

        [[-4.34426184e-05, -1.30646653e-04, -1.22757093e-04, ...,
           2.60653283e-04,  2.94846914e-04,  1.81649205e-04],
         [ 3.92325481e-05, -5.29975819e-05, -4.86435560e-05, ...,
           9.54413623e-07,  1.33914176e-05,  7.47919609e-05],
         [ 3.10389872e-05, -4.99452927e-05, -3.86961868e-05, ...,
           6.24279163e-06,  1.50812117e-05,  6.73406234e-05],
...
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan]],

        [[            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         ...,
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan]]]])
Coordinates:
  * time     (time) datetime64[ns] 2012-01-01 ... 2012-01-01T23:00:00
  * k        (k) int32 0 1 2 3 4 5 6 7 8 9 10 ... 78 79 80 81 82 83 84 85 86 87
  * j_g      (j_g) float32 0.0 1.0 2.0 3.0 4.0 ... 390.0 391.0 392.0 393.0 394.0
  * i        (i) float32 0.0 1.0 2.0 3.0 4.0 ... 475.0 476.0 477.0 478.0 479.0

In [ ]:
%%time
# --- compute vorticity using xgcm and interpolate to X, Y
        # see https://xgcm.readthedocs.io/en/latest/xgcm-examples/02_mitgcm.htm
#vorticity = (grid.diff(ds.V*ds.DXG, 'X') - grid.diff(ds.U*ds.DYG, 'Y'))/ds.RAZ
vorticity = grid.interp(grid.interp(vorticity, 'X', boundary='extend'), 'Y', boundary='extend')

In [ ]:
%%time
# --- compute vorticity using xgcm and interpolate to X, Y
        # see https://xgcm.readthedocs.io/en/latest/xgcm-examples/02_mitgcm.htm
#vorticity = (grid.diff(ds.V*ds.DXG, 'X') - grid.diff(ds.U*ds.DYG, 'Y'))/ds.RAZ
vorticity = grid.interp(vorticity, 'Y', boundary='extend')


In [11]:
# loop through files, then compute steric height, vorticity, etc. on the i/j grid
# fis = range(1)
fis = range(len(fg))
yn = 'y'
for fi in fis:
    # --- select data ---
    thisf=fg[fi]
    
    # check if output file already exists
    fnout = thisf.replace(RegionName + '_' , RegionName + '_derived-fields_')
    fnout = fnout.replace(RegionName + '/' , RegionName + '/derived/')
    
    if (os.path.isfile(fnout) & (yn.lower() == 'y')):
        yn = input(f'\n{fnout} already exists. Overwrite? (this decision will apply to all files) [y/N]')
    if (os.path.isfile(fnout) & (yn.lower() == 'n')):
        # do nothing
        1
    elif (yn.lower() == 'y') | (not(os.path.isfile(fnout))):   
        print(thisf , '(' , fi+1, 'of', len(fis), ')')  
        ds = xr.open_dataset(thisf)

        # create datasets for variables of interest:
        ss = ds.Salt
        tt = ds.Theta
        pp = xr.DataArray(sw.p_from_z(ds.Z,ds.YC))

        # --- compute steric height in steps ---
        # 1. compute absolute salinity and conservative temperature
        sa = xr.apply_ufunc(sw.SA_from_SP, ss, pp, xav, yav, dask='parallelized', output_dtypes=[ss.dtype])
        sa.compute()
        ct = xr.apply_ufunc(sw.CT_from_pt, sa, tt, dask='parallelized', output_dtypes=[ss.dtype])
        ct.compute()
        dd = xr.apply_ufunc(sw.density.rho, sa, ct, pp, dask='parallelized', output_dtypes=[ss.dtype])
        dd.compute()
        # 2. compute specific volume anomaly: gsw.density.specvol_anom_standard(SA, CT, p)
        sva = xr.apply_ufunc(sw.density.specvol_anom_standard, sa, ct, pp, dask='parallelized', output_dtypes=[ss.dtype])
        sva.compute()
        # 3. compute steric height = integral(0:z1) of Dref(z)*sva(z)*dz(z)
        # - first, interpolate Dref to the model pressure levels
        Drefi = Dref.interp(LEV=-ds.Z)
        dz = -ds.Z_bnds.diff(dim='nb').drop_vars('nb').squeeze() # distance between interfaces

        # steric height computation (summation/integral)
        # - increase the size of Drefi and dz to match the size of sva
        Db = Drefi.broadcast_like(sva)
        dzb = dz.broadcast_like(sva)
        dum = Db * sva * dzb
        sh = dum.cumsum(dim='k')

        # --- compute vorticity using xgcm and interpolate to X, Y
        # see https://xgcm.readthedocs.io/en/latest/xgcm-examples/02_mitgcm.html
        #vorticity = (grid.diff(ds.V*ds.DXG, 'X') - grid.diff(ds.U*ds.DYG, 'Y'))/ds.RAZ
        #vorticity = grid.interp(grid.interp(vorticity, 'X', boundary='extend'), 'Y', boundary='extend')

        # --- save derived fields in a new file
        # - convert sh and zeta to datasets
        #dout = vorticity.to_dataset(name='vorticity')
        sh_ds = sh.to_dataset(name='steric_height')
        dout = dout.merge(sh_ds)
        # add/rename the Argo reference profile variables
        tref = Tref.to_dataset(name='Tref')
        tref = tref.merge(Sref).rename({'SALT': 'Sref'}).\
            rename({'LEV':'zref','latitude':'yav','longitude':'xav'}).\
            drop_vars({'i','j'})
        # - add ref profiles to dout and drop uneeded vars/coords
        dout = dout.merge(tref).drop_vars({'longitude','latitude','LEV','i','j'})

        # - save netcdf file with derived fields
        netcdf_fill_value = nc4.default_fillvals['f4']
        dv_encoding = {}
        for dv in dout.data_vars:
            dv_encoding[dv]={'zlib':True,  # turns compression on\
                        'complevel':9,     # 1 = fastest, lowest compression; 9=slowest, highest compression \
                        'shuffle':True,    # shuffle filter can significantly improve compression ratios, and is on by default \
                        'dtype':'float32',\
                        '_FillValue':netcdf_fill_value}
        # save to a new file
        print(' ... saving to ', fnout)
        dout.to_netcdf(fnout,format='netcdf4',encoding=dv_encoding)


/home/manjaree/Documents/LLC4320_pre-SWOT_10_days/ACC_SMST/osse_model_input/LLC4320_pre-SWOT_ACC_SMST_20120101.nc ( 1 of 10 )


NameError: name 'dout' is not defined